In [1]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

import json

In [2]:
from konlpy.tag import Okt
okt = Okt()

In [3]:
openFileName = '전자담배 효과_naver_news_1000.json'
    
f = open(openFileName, 'r', encoding='utf-8').read()

jsonData = json.loads(f)
data = ''

for item in jsonData:
    if 'description' in item.keys():
        data = data + re.sub(r'[^\w]', ' ', item['description']) + ' '
        
# f.close()

In [4]:
def tokenizer(raw, pos=["Noun"], stopword=[]): # 다른 품사 추가 원할 시 pos=["Noun", "Josa", ...]
    return [
        word for word, tag in okt.pos(
            raw, 
            norm=True,   # normalize 그랰ㅋㅋ -> 그래ㅋㅋ
            stem=True    # stemming 바뀌나->바뀌다
            )
            if len(word) > 1 and tag in pos and word not in stopword
        ]

documents1 = tokenizer(data)

In [5]:
import gensim

documents2 = [documents1] # gensim.corpora.Dictionary() 는 매개변수로 리스트 타입을 받기 때문에
dictionary = gensim.corpora.Dictionary(documents2)
print(dictionary.token2id)
print('dictionary size : %d' % len(dictionary)) # dictionary size : 37987

{'가가': 0, '가게': 1, '가격': 2, '가계': 3, '가공': 4, '가공비': 5, '가까이': 6, '가능': 7, '가능성': 8, '가량': 9, '가속': 10, '가수': 11, '가연성': 12, '가열': 13, '가운데': 14, '가율': 15, '가이드': 16, '가이드라인': 17, '가입': 18, '가장': 19, '가전제품': 20, '가정': 21, '가정의학': 22, '가중': 23, '가지': 24, '가축': 25, '가치': 26, '가파르': 27, '가해': 28, '가향': 29, '각각': 30, '각광': 31, '각국': 32, '각심': 33, '각종': 34, '간과': 35, '간담': 36, '간섭': 37, '간접': 38, '간접흡연': 39, '간질': 40, '간편식': 41, '갈등': 42, '갈수록': 43, '감고': 44, '감사원': 45, '감소': 46, '감안': 47, '감지': 48, '감축': 49, '갑지': 50, '강남': 51, '강당': 52, '강도': 53, '강북': 54, '강사': 55, '강세': 56, '강연': 57, '강의': 58, '강이슬': 59, '강제': 60, '강조': 61, '강화': 62, '개국': 63, '개념': 64, '개막전': 65, '개발': 66, '개발자': 67, '개별': 68, '개비': 69, '개사': 70, '개선': 71, '개설': 72, '개시': 73, '개월': 74, '개인': 75, '개인정보': 76, '개입': 77, '개점': 78, '개정안': 79, '개피': 80, '객수': 81, '거꾸로': 82, '거나': 83, '거래': 84, '거래소': 85, '거론': 86, '거리': 87, '거부': 88, '거울': 89, '거의': 90, '거점': 91, '거짓': 92, '거치': 93, '걱정': 94, '건강': 95, '건조': 96, '검거': 97, '검

In [6]:
corpus_path = '전자담배 효과_naver_news_1000.json'

class Documents:
    def __init__(self, path):
        self.path = path
    def __iter__(self):
        with open(self.path, encoding='utf-8') as f:
            for doc in f:
                yield doc.strip().split()

documents = Documents(corpus_path)

In [7]:
class Corpus:
    def __init__(self, path, dictionary):
        self.path = path
        self.dictionary = dictionary
        self.length = 0
    def __iter__(self):
        with open(self.path, encoding='utf-8') as f:
            for doc in f:
                yield self.dictionary.doc2bow(doc.split())
    def __len__(self):
        if self.length == 0:
            with open(self.path, encoding='utf-8') as f:
                for i, doc in enumerate(f):
                    continue
            self.length = i + 1
        return self.length

corpus = Corpus(corpus_path, dictionary)
for i, doc in enumerate(corpus):
    #if i >= 5: break
    print(doc)

[]
[]
[(197, 1), (486, 1), (506, 1), (893, 1), (2086, 1)]
[]
[]
[]
[(327, 1), (754, 1), (1675, 1)]
[]
[]
[(170, 1), (297, 1), (645, 1), (718, 1), (1275, 1)]
[]
[]
[]
[(170, 1), (645, 1)]
[]
[]
[(251, 1), (344, 1), (2203, 1)]
[]
[]
[]
[]
[]
[]
[(288, 1), (736, 1), (801, 1), (977, 1), (999, 1), (1001, 1), (1099, 1), (1421, 1), (1682, 1), (1709, 1), (1851, 1), (2095, 1), (2133, 1), (2188, 1), (2341, 1)]
[]
[]
[]
[]
[]
[]
[(483, 1), (645, 1), (1864, 1), (2247, 1)]
[]
[]
[]
[]
[]
[]
[(303, 1), (931, 1)]
[]
[]
[]
[]
[]
[]
[(251, 2), (332, 1), (723, 1), (1141, 1), (1151, 1), (1897, 1)]
[]
[]
[]
[]
[]
[]
[(288, 1), (736, 1), (801, 1), (977, 1), (999, 1), (1001, 1), (1099, 1), (1421, 1), (1682, 1), (1709, 1), (1851, 1), (2095, 1), (2133, 1), (2188, 1), (2341, 1)]
[]
[]
[]
[]
[]
[]
[(251, 1), (344, 1), (2203, 1)]
[]
[]
[]
[]
[]
[]
[(203, 1), (1965, 1)]
[]
[]
[]
[]
[]
[]
[(1215, 1), (1382, 1), (2236, 1)]
[]
[]
[]
[(404, 1)]
[]
[]
[(288, 1), (736, 1), (801, 1), (977, 1), (999, 1), (1001, 1), (1099

[]
[(196, 1), (620, 1), (900, 1), (1421, 1)]
[]
[]
[]
[(196, 1)]
[]
[]
[(967, 1), (1153, 1), (1308, 1)]
[]
[]
[]
[(968, 1)]
[]
[]
[(447, 1), (1421, 1), (1474, 1)]
[]
[]
[]
[]
[]
[]
[(484, 1), (942, 1), (1942, 1)]
[]
[]
[]
[(55, 1), (811, 1), (2213, 1)]
[]
[]
[(321, 1), (932, 1), (967, 1), (1348, 1)]
[]
[]
[]
[]
[]
[]
[(321, 1), (483, 1), (1169, 1), (1363, 1)]
[]
[]
[]
[(809, 1), (1116, 1), (1670, 1)]
[]
[]
[(336, 1), (432, 1), (483, 1), (507, 1), (1116, 1), (1658, 1), (2324, 1)]
[]
[]
[]
[(432, 1), (679, 1)]
[]
[]
[(447, 1), (1421, 1), (1474, 1)]
[]
[]
[]
[(196, 1), (900, 1)]
[]
[]
[(447, 1), (1421, 1), (1474, 1)]
[]
[]
[]
[(196, 1), (900, 1)]
[]
[]
[(1421, 1), (1557, 1)]
[]
[]
[]
[(196, 1), (900, 1)]
[]
[]
[(196, 1), (870, 1), (1421, 1)]
[]
[]
[]
[(196, 1), (900, 1)]
[]
[]
[(1421, 1)]
[]
[]
[]
[(900, 1)]
[]
[]
[(447, 1), (1421, 1), (1474, 1)]
[]
[]
[]
[]
[]
[]
[(196, 1), (870, 1), (1295, 1), (1714, 1), (1920, 1)]
[]
[]
[]
[(196, 1)]
[]
[]
[(447, 2), (484, 1), (1421, 1)]
[]
[]
[]
[(88,

[(760, 1)]
[]
[]
[(30, 1), (303, 1), (426, 1), (428, 1), (502, 1)]
[]
[]
[]
[(426, 1), (1503, 1)]
[]
[]
[(203, 2), (303, 2), (426, 1), (754, 1)]
[]
[]
[]
[]
[]
[]
[(14, 1), (136, 1), (303, 1), (1489, 1), (1927, 1), (2086, 1), (2198, 1)]
[]
[]
[]
[(303, 1)]
[]
[]
[(760, 1), (2226, 1)]
[]
[]
[]
[(2049, 1)]
[]
[]
[(303, 1), (428, 1), (2048, 1)]
[]
[]
[]
[]
[]
[]
[(303, 2), (428, 1), (506, 1), (1470, 1)]
[]
[]
[]
[]
[]
[]
[(426, 1), (494, 1), (548, 1), (588, 1), (658, 1), (1162, 1), (1575, 1), (2187, 1)]
[]
[]
[]
[(303, 1), (426, 1), (494, 1)]
[]
[]
[(303, 2), (428, 1), (506, 1), (1470, 1)]
[]
[]
[]
[(426, 1)]
[]
[]
[(171, 1), (303, 1), (343, 1), (426, 1), (502, 1), (588, 1), (597, 1), (658, 1), (723, 1), (936, 1), (1329, 1), (1797, 1), (2052, 1), (2110, 1)]
[]
[]
[]
[(343, 1), (426, 1)]
[]
[]
[(506, 1), (1557, 1), (1877, 1)]
[]
[]
[]
[(2193, 1)]
[]
[]
[(506, 1), (892, 1), (1998, 1)]
[]
[]
[]
[(216, 1), (2193, 1), (2283, 1)]
[]
[]
[(251, 1), (343, 1), (790, 1)]
[]
[]
[]
[(908, 1), (1011, 1

[]
[(203, 1), (426, 1), (778, 1), (1435, 1), (1831, 1), (1906, 1)]
[]
[]
[]
[]
[]
[]
[(19, 1), (2199, 1)]
[]
[]
[]
[(456, 1), (1098, 1)]
[]
[]
[(8, 1), (30, 1), (203, 1), (426, 1), (517, 1), (778, 1), (1435, 1), (1831, 1), (1906, 1)]
[]
[]
[]
[(124, 1)]
[]
[]
[(925, 1), (1811, 1), (2358, 1)]
[]
[]
[]
[(456, 1)]
[]
[]
[(203, 1), (426, 1), (778, 1), (1435, 1), (1831, 1)]
[]
[]
[]
[(1671, 1)]
[]
[]
[(124, 1), (676, 1)]
[]
[]
[]
[]
[]
[]
[(506, 1), (1340, 1), (2362, 1)]
[]
[]
[]
[]
[]
[]
[(19, 1), (1032, 1), (2199, 1), (2277, 1)]
[]
[]
[]
[]
[]
[]
[(426, 1), (778, 1), (1435, 1), (1831, 1)]
[]
[]
[]
[(1671, 1)]
[]
[]
[(124, 1)]
[]
[]
[]
[(779, 1)]
[]
[]
[(129, 1), (203, 1), (426, 1), (778, 1), (1435, 1), (1831, 1), (1906, 1)]
[]
[]
[]
[(676, 1), (713, 1)]
[]
[]
[(19, 1), (1032, 1), (2199, 1), (2277, 1)]
[]
[]
[]
[(456, 1)]
[]
[]
[(19, 1), (1032, 1), (2199, 1), (2277, 1)]
[]
[]
[]
[(456, 1)]
[]
[]
[(124, 1), (517, 1), (943, 1), (1862, 1), (2165, 1), (2362, 1)]
[]
[]
[]
[(456, 1), (1098, 1)]


[(14, 1), (124, 1), (433, 1), (506, 1), (1346, 1)]
[]
[]
[]
[]
[]
[]
[(124, 1), (506, 1), (1817, 1), (2003, 1)]
[]
[]
[]
[(943, 1)]
[]
[]
[(124, 1), (426, 1), (506, 1), (1817, 1), (1831, 1), (2277, 1)]
[]
[]
[]
[]
[]
[]
[(124, 1), (506, 1), (1817, 1), (2003, 1)]
[]
[]
[]
[(2362, 1)]
[]
[]
[(124, 1), (506, 1), (1817, 1), (2003, 1)]
[]
[]
[]
[(384, 1), (1671, 1)]
[]
[]
[(506, 1), (1817, 1)]
[]
[]
[]
[]
[]
[]
[(124, 1), (506, 1), (1817, 1), (2003, 1)]
[]
[]
[]
[(1241, 1)]
[]
[]
[(124, 1), (1817, 1), (2003, 1)]
[]
[]
[]
[(2003, 1)]
[]
[]
[(124, 1), (343, 1), (1817, 1)]
[]
[]
[]
[(943, 1), (1241, 1)]
[]
[]
[(124, 1), (278, 1), (426, 1), (506, 1), (1817, 1), (2003, 1)]
[]
[]
[]
[(384, 1)]
[]
[]
[(14, 1), (124, 1), (426, 1), (506, 1), (1817, 1), (1831, 1), (2277, 1)]
[]
[]
[]
[(1241, 1)]
[]
[]
[(124, 1), (506, 1), (1817, 1), (2003, 1)]
[]
[]
[]
[(943, 1), (1241, 1)]
[]
[]
[(124, 1), (506, 1), (1811, 1), (2003, 1), (2358, 1)]
[]
[]
[]
[(1098, 1), (1241, 1)]
[]
[]
[(124, 1), (506, 1), (1817, 1)

[]
[]
[(130, 1), (405, 1), (1922, 1)]
[]
[]
[(874, 1), (2133, 1), (2203, 1)]
[]
[]
[]
[(1191, 1)]
[]
[]
[(506, 1), (948, 1), (1340, 1), (2319, 1)]
[]
[]
[]
[(255, 1)]
[]
[]
[(506, 1), (2226, 1)]
[]
[]
[]
[(1336, 1)]
[]
[]
[(645, 1), (2133, 1), (2203, 1)]
[]
[]
[]
[]
[]
[]
[(506, 1), (948, 1)]
[]
[]
[]
[(432, 1)]
[]
[]
[(506, 1), (948, 1)]
[]
[]
[]
[]
[]
[]
[(506, 1), (948, 1)]
[]
[]
[]
[(2056, 1), (2229, 1)]
[]
[]


In [8]:
from gensim.models import LdaModel

lda_model = LdaModel(corpus, id2word=dictionary, num_topics=10)
print(lda_model)

LdaModel(num_terms=2371, num_topics=10, decay=0.5, chunksize=2000)


In [9]:
def get_topic_term_prob(lda_model):
    topic_term_freqs = lda_model.state.get_lambda()
    topic_term_prob = topic_term_freqs / topic_term_freqs.sum(axis=1)[:, None]
    return topic_term_prob

In [10]:
print(lda_model.alpha.shape) # (n_topics,)
print(lda_model.alpha.sum()) # 1.0

topic_term_prob = get_topic_term_prob(lda_model)
print(topic_term_prob.shape)     # (n_topics, n_terms)
print(topic_term_prob[0].sum())  # 1.0

(10,)
1.0
(10, 2371)
1.0


In [11]:
import pyLDAvis.gensim as gensimvis

prepared_data = gensimvis.prepare(lda_model, corpus, dictionary)

C:\Users\BOGYUNG\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [12]:
pyLDAvis.display(prepared_data)
# pyLDAvis.show(prepared_data)